# Contents <a name="toc"></a>
1. [Default](#p1)
2. [Integration with MLFLOW](#p2)
3. [Multitarget](#p3)
4. [Command line mode](#p4)
5. [Optuna](#p5)


In [1]:
def dataset_info(data):
    n_features = data.x.shape[1]
    n_nodes = data.x.shape[0]
    n_edges = data.edge_index.shape[1]
    if len(data.y.shape) == 1:
        print(f'# nodes    {n_nodes} \n# features {n_features} \n# edges    {n_edges} \n# classes  {len(data.y.unique())}')
    else:
        print(f'# nodes    {n_nodes} \n# features {n_features} \n# edges    {n_edges} \n# tasks    {data.y.shape[1]}')

# 1. Default <a name="p1"></a>

In [2]:
from cool_graph.runners import Runner
from torch_geometric import datasets
import torch
import pandas as pd

/home/dapavlikov/.conda/envs/cool_graph_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# use simple Amazon dataset with Computers
dataset = datasets.Amazon(root='./data/Amazon', name='Computers')
data = dataset.data
data

Data(x=[13752, 767], edge_index=[2, 491722], y=[13752])

In [4]:
dataset_info(data)

# nodes    13752 
# features 767 
# edges    491722 
# classes  10


In [5]:
# lets split data before training (also split data method include in Runner, so you dont have to do this step)
runner = Runner(data)

In [6]:
loaders = runner.sample_data()

Sample data: 100%|██████████| 14/14 [00:01<00:00, 13.36it/s]


In [7]:
loaders.keys()

dict_keys(['train_loader', 'test_loader'])

In [8]:
loaders["train_loader"][:5]

[Data(x=[10219, 767], edge_index=[2, 48223], y=[10219], group_mask=[10219], label_mask=[10219], input_id=[250], batch_size=250),
 Data(x=[10038, 767], edge_index=[2, 44787], y=[10038], group_mask=[10038], label_mask=[10038], input_id=[250], batch_size=250),
 Data(x=[10115, 767], edge_index=[2, 45012], y=[10115], group_mask=[10115], label_mask=[10115], input_id=[250], batch_size=250),
 Data(x=[10190, 767], edge_index=[2, 47636], y=[10190], group_mask=[10190], label_mask=[10190], input_id=[250], batch_size=250),
 Data(x=[10129, 767], edge_index=[2, 45880], y=[10129], group_mask=[10129], label_mask=[10129], input_id=[250], batch_size=250)]

In [9]:
%%time
# training
result = runner.run(train_loader=loaders["train_loader"],
                    test_loader=loaders["test_loader"])

2024-06-04 14:06:09.886 | INFO     | cool_graph.train.helpers:eval_epoch:209 - test:
 {'accuracy': 0.533, 'cross_entropy': 1.386, 'f1_weighted': 0.506, 'calc_time': 0.004, 'main_metric': 0.533}
2024-06-04 14:06:10.674 | INFO     | cool_graph.train.helpers:eval_epoch:209 - train:
 {'accuracy': 0.546, 'cross_entropy': 1.369, 'f1_weighted': 0.517, 'calc_time': 0.013, 'main_metric': 0.546}
2024-06-04 14:06:17.035 | INFO     | cool_graph.train.helpers:eval_epoch:209 - test:
 {'accuracy': 0.883, 'cross_entropy': 0.358, 'f1_weighted': 0.868, 'calc_time': 0.006, 'main_metric': 0.883}
2024-06-04 14:06:17.635 | INFO     | cool_graph.train.helpers:eval_epoch:209 - train:
 {'accuracy': 0.91, 'cross_entropy': 0.272, 'f1_weighted': 0.895, 'calc_time': 0.01, 'main_metric': 0.91}
2024-06-04 14:06:24.081 | INFO     | cool_graph.train.helpers:eval_epoch:209 - test:
 {'accuracy': 0.913, 'cross_entropy': 0.301, 'f1_weighted': 0.912, 'calc_time': 0.005, 'main_metric': 0.913}
2024-06-04 14:06:24.746 | INFO 

CPU times: user 27min 17s, sys: 1min 16s, total: 28min 34s
Wall time: 33.5 s


In [10]:
result['best_loss']["accuracy"]

0.913

In [11]:
# local logging 
str(runner.chkpt_dir)

'checkpoints/2024-06-04 14:06:00'

In [20]:
# getting predictionsm (probs for all labels (from 0 to 9))
result["train_preds"]

{'y': array([[1.71531334e-09, 3.22356577e-08, 5.37139421e-15, ...,
         3.12158238e-15, 6.19763885e-09, 2.53535966e-16],
        [2.28220756e-06, 2.24519881e-05, 9.97316778e-01, ...,
         1.49518615e-04, 4.87086709e-07, 1.38805498e-13],
        [4.34640697e-07, 2.57379119e-03, 3.86692989e-08, ...,
         1.10065708e-08, 9.93130863e-01, 3.90740382e-08],
        ...,
        [1.37076136e-12, 3.96369396e-11, 1.81455073e-20, ...,
         1.63082729e-20, 1.29661898e-11, 5.83262216e-21],
        [1.04628584e-09, 2.02045940e-05, 2.15618096e-11, ...,
         1.01657335e-10, 9.89686430e-01, 7.48045270e-09],
        [1.67321905e-05, 3.07874376e-04, 9.87183869e-01, ...,
         7.11549248e-04, 1.97213558e-05, 4.04881684e-11]], dtype=float32)}

In [21]:
result["test_preds"]

{'y': array([[1.9071331e-05, 5.5390149e-03, 2.8504763e-07, ..., 2.3533222e-07,
         3.0775595e-01, 4.6698706e-06],
        [1.8434193e-08, 2.9395288e-04, 8.7993174e-10, ..., 3.0116459e-10,
         9.9836153e-01, 4.0763761e-09],
        [6.1273308e-06, 5.1038620e-05, 9.9257654e-01, ..., 1.8885201e-04,
         3.8187900e-07, 5.0805284e-14],
        ...,
        [2.6789903e-12, 1.3569391e-09, 1.5660323e-10, ..., 1.6329977e-06,
         1.0652494e-07, 4.3612977e-06],
        [1.8761559e-08, 1.5271330e-03, 2.1465221e-09, ..., 1.5873597e-09,
         9.9539715e-01, 7.7008888e-09],
        [5.9854602e-07, 6.2096649e-07, 5.6886674e-12, ..., 3.3290549e-12,
         1.1345562e-07, 2.2665092e-13]], dtype=float32)}

### getting embedding 

In [22]:
runner_emb = Runner(data, embedding_data=True)

In [23]:
data

Data(x=[13752, 767], edge_index=[2, 491722], y=[13752], group_mask=[13752], label_mask=[13752])

In [24]:
%%time
# running without sample data (automatically in runner)
result = runner_emb.run()

Sample data: 100%|██████████| 14/14 [00:01<00:00, 13.94it/s]
2024-06-04 14:08:08.328 | INFO     | cool_graph.train.helpers:eval_epoch:209 - test:
 {'accuracy': 0.613, 'cross_entropy': 1.196, 'f1_weighted': 0.52, 'calc_time': 0.004, 'main_metric': 0.613}
2024-06-04 14:08:09.003 | INFO     | cool_graph.train.helpers:eval_epoch:209 - train:
 {'accuracy': 0.599, 'cross_entropy': 1.212, 'f1_weighted': 0.504, 'calc_time': 0.011, 'main_metric': 0.599}
2024-06-04 14:08:15.570 | INFO     | cool_graph.train.helpers:eval_epoch:209 - test:
 {'accuracy': 0.903, 'cross_entropy': 0.305, 'f1_weighted': 0.902, 'calc_time': 0.005, 'main_metric': 0.903}
2024-06-04 14:08:16.510 | INFO     | cool_graph.train.helpers:eval_epoch:209 - train:
 {'accuracy': 0.918, 'cross_entropy': 0.249, 'f1_weighted': 0.917, 'calc_time': 0.016, 'main_metric': 0.918}
2024-06-04 14:08:23.104 | INFO     | cool_graph.train.helpers:eval_epoch:209 - test:
 {'accuracy': 0.921, 'cross_entropy': 0.275, 'f1_weighted': 0.921, 'calc_time

CPU times: user 31min 5s, sys: 1min 27s, total: 32min 32s
Wall time: 36.3 s


In [29]:
# embeddings for train split

result["train_preds"]["emb"]

array([[-2.9838342e-02, -8.2892962e-03, -1.1445961e-02, ...,
        -2.6091158e-01, -9.7843327e-02, -3.3539826e-01],
       [ 3.5486307e+00, -1.3963007e-02,  5.2889719e+00, ...,
         6.0339184e+00, -7.9321809e-02, -4.5517709e-02],
       [-2.6562642e-02, -1.8376589e-02, -5.5455822e-03, ...,
        -2.9938933e-01,  9.1956482e+00, -2.2355743e-01],
       ...,
       [-3.8466842e-03,  2.2067773e-01, -3.0117228e-03, ...,
        -1.6229012e-03, -5.7519055e-03, -1.1865041e-02],
       [ 8.4885913e-01,  4.4799657e+00,  4.2979140e+00, ...,
        -1.7672050e-01, -6.7367278e-02, -1.3604055e-01],
       [-2.8954877e-02, -1.5898040e-02, -1.0180769e-02, ...,
        -3.0768555e-01, -1.3888237e-01, -4.3958679e-01]], dtype=float32)

In [30]:
# embeddings for test split

result["test_preds"]["emb"]

array([[ 3.8058176e+00, -3.5299242e-02,  6.4857578e+00, ...,
         9.5812654e+00, -1.3134895e-01, -8.3783865e-02],
       [ 2.5830977e+00,  5.0719538e+00,  7.0205364e+00, ...,
        -1.4936209e-01, -5.3882025e-02, -8.6760461e-02],
       [ 1.3296409e+00,  5.7701249e+00,  5.6685710e+00, ...,
        -7.8421131e-02, -5.7891291e-02, -7.3489815e-02],
       ...,
       [-4.4046521e-02, -1.1343454e-02, -1.7138092e-02, ...,
        -2.7405626e-01, -1.0182064e-01, -3.7270886e-01],
       [-7.5560897e-03, -1.3696783e-02,  4.8613542e-01, ...,
        -2.9631352e-02,  1.5251670e+00, -2.0398382e-02],
       [ 6.1667042e+00, -3.7080124e-02,  6.3836689e+00, ...,
        -1.4345302e-01, -3.9000323e-01, -2.3159283e-01]], dtype=float32)

# 2. Integration with MLFLOW <a name="p2"></a>

In [20]:
# using MLflow

In [21]:
dataset = datasets.Amazon(root = './data/Amazon',name='Computers')
data = dataset.data

In [22]:
dataset_info(data)

# nodes    13752 
# features 767 
# edges    491722 
# classes  10


In [29]:
# fill the data of MLFlow in config logging/in_memory_data or if you use cli mode - logging/default

# 3. Multitarget <a name="p3"></a>

In [33]:
from torch_geometric import datasets
from cool_graph.runners import Runner

dataset = datasets.Yelp(root='../data/Yelp')
data = dataset.data
data.y = data.y.long()

In [33]:
dataset_info(data)

# nodes    716847 
# features 300 
# edges    13954819 
# tasks    100


In [23]:
# you can override default params 

In [37]:
config_path = "../cool_graph/config/in_memory_data.yaml"

In [38]:
runner = Runner(data, metrics=['roc_auc','accuracy', 'f1'], batch_size='auto', train_size=0.1, test_size=0.02,
                overrides=['training.n_epochs=1'], config_path=config_path)

In [39]:
%%time
results = runner.run()

Sample data: 100%|██████████| 10/10 [00:05<00:00,  1.86it/s]
2024-06-04 14:12:57.596 | INFO     | cool_graph.train.helpers:eval_epoch:209 - test:
 {'roc_auc': 0.711, 'accuracy': 0.911, 'f1': 0.009, 'calc_time': 0.083, 'main_metric': 0.711}
2024-06-04 14:13:21.609 | INFO     | cool_graph.train.helpers:eval_epoch:209 - train:
 {'roc_auc': 0.711, 'accuracy': 0.911, 'f1': 0.008, 'calc_time': 0.4, 'main_metric': 0.711}


CPU times: user 50min 57s, sys: 4min 25s, total: 55min 22s
Wall time: 1min 29s


In [40]:
results["best_loss"]["roc_auc"]

0.711

# 4. Command line mode <a name="p4"></a>

- feats on edges - training attention nn
- 2+ groups in nodes with different feats on it
- multitarget
- categorical feats 

In [36]:
!head -n 25 ../cool_graph/config/data/default.yaml

# this config is using  for data where nodes have groups
# path to train/validation data in parquet
train:
  nodes_path: ../../tests/sample_data/nodes.parquet
  edges_path: ../../tests/sample_data/edges.parquet
  labels_path: ../../tests/sample_data/labels.parquet
validation:
  nodes_path: ../../tests/sample_data/nodes.parquet
  edges_path: ../../tests/sample_data/edges.parquet
  labels_path: ../../tests/sample_data/labels.parquet
read_edge_attr: True # set True if you read data from disk and it has feats on edges
group_mask_col: node_feature_1 # column for mask in groups
label_mask_col: label_2 # column for mask in labels
label_index_col: index # index column in labels data
# columns with indices (edges between nodes)
edge_index_cols: 
  - index2
  - index1
# names in node groups 
groups_names: 
  1: group_2
  0: group_1
# target columns 
label_cols: 
  - index


In [41]:
coolrun --config /cool_graph/config/data/full.yaml 

# 5. Optuna <a name="p5"></a>

 - **Searching the best params for the models**
 
 - HyperRunner:
 * 1st trial - deafault params from config 
 * 2nd trial - you can add own trial as argument enqueue_trial in optimazire_run method, and next trial optuna optimize model params randomly, if None -> randomly after 1st default trial
 
 returns a DataFrame with trials and metrics 

In [42]:
from cool_graph.runners import HypeRunner
from torch_geometric import datasets

In [43]:
data = datasets.Amazon(root='./data/Amazon/', name='Computers').data

In [44]:
runner = HypeRunner(data)

In [45]:
# own dict with model params for trial
my_params1 = {'conv_type': 'GraphConv',
 'activation': 'relu',
 'lin_prep_dropout_rate': 0.4,
 'lin_prep_len': 1,
 'lin_prep_size_common': 512,
 'lin_prep_sizes': [256],
 'lin_prep_weight_norm_flag': True,
 'graph_conv_weight_norm_flag': True,
 'n_hops': 2,
 'conv1_aggrs': {'mean': 64, 'max': 32, 'add': 16},
 'conv1_dropout_rate': 0.2,
 'conv2_aggrs': {'mean': 32, 'max': 16, 'add': 8},
 'conv2_dropout_rate': 0.2}

my_params2 = {'conv_type': 'GraphConv',
 'activation': 'prelu',
 'lin_prep_dropout_rate': 0.5,
 'lin_prep_len': 1,
 'lin_prep_size_common': 512,
 'lin_prep_sizes': [256],
 'lin_prep_weight_norm_flag': False,
 'graph_conv_weight_norm_flag': True,
 'n_hops': 2,
 'conv1_aggrs': {'mean': 64, 'max': 32, 'add': 16},
 'conv1_dropout_rate': 0.2,
 'conv2_aggrs': {'mean': 32, 'max': 16, 'add': 8},
 'conv2_dropout_rate': 0.2}

In [46]:
result = runner.optimize_run(n_trials=5, enqueue_trial=[my_params1, my_params2])

Sample data: 100%|██████████| 14/14 [00:01<00:00, 10.15it/s]
[I 2023-11-20 11:04:50,648] A new study created in memory with name: no-name-b21ba7cf-0d18-49fb-9b2d-0860c880a72d
/data/home/dapavlikov/coolgraph/cool_graph/runners.py:659: ExperimentalWarning: enqueue_trial is experimental (supported from v1.2.0). The interface can change in the future.
  self.study.enqueue_trial(trial_params)
/home/dapavlikov/.conda/envs/cool_graph_env2_py38/lib/python3.8/site-packages/optuna/study/study.py:857: ExperimentalWarning: create_trial is experimental (supported from v2.0.0). The interface can change in the future.
  create_trial(state=TrialState.WAITING, system_attrs={"fixed_params": params})
/home/dapavlikov/.conda/envs/cool_graph_env2_py38/lib/python3.8/site-packages/optuna/study/study.py:856: ExperimentalWarning: add_trial is experimental (supported from v2.0.0). The interface can change in the future.
  self.add_trial(
/data/home/dapavlikov/coolgraph/cool_graph/runners.py:666: ExperimentalWar

Study statistics: 
  Number of finished trials:  5
  Number of complete trials:  5
Best trial:
  Value:  0.916
  Params: 
{'conv_type': 'GraphConv', 'activation': 'leakyrelu', 'lin_prep_len': 1, 'lin_prep_dropout_rate': 0.4, 'lin_prep_weight_norm_flag': True, 'lin_prep_size_common': 512, 'lin_prep_sizes': [256], 'n_hops': 2, 'conv1_aggrs': {'mean': 128, 'max': 64, 'add': 32}, 'conv1_dropout_rate': 0.2, 'conv2_aggrs': {'mean': 64, 'max': 32, 'add': 16}, 'conv2_dropout_rate': 0.2, 'graph_conv_weight_norm_flag': True}


In [47]:
result

,number,value,datetime_start,datetime_complete,duration,system_attrs_fixed_params,state,conv_type,activation,lin_prep_len,lin_prep_dropout_rate,lin_prep_weight_norm_flag,lin_prep_size_common,lin_prep_sizes,n_hops,conv1_aggrs,conv1_dropout_rate,conv2_aggrs,conv2_dropout_rate,graph_conv_weight_norm_flag
0,0,0.916,2023-11-20 11:04:50.651106,2023-11-20 11:05:23.925253,0 days 00:00:33.274147,"{'activation': 'leakyrelu', 'lin_prep_len': 1,...",COMPLETE,GraphConv,leakyrelu,1,0.4,True,512,[256],2,"{'mean': 128, 'max': 64, 'add': 32}",0.2,"{'mean': 64, 'max': 32, 'add': 16}",0.2,True
1,1,0.913,2023-11-20 11:05:23.933594,2023-11-20 11:05:55.882016,0 days 00:00:31.948422,"{'activation': 'relu', 'lin_prep_len': 1, 'lin...",COMPLETE,GraphConv,relu,1,0.4,True,512,[256],2,"{'mean': 64, 'max': 32, 'add': 16}",0.2,"{'mean': 32, 'max': 16, 'add': 8}",0.2,True
2,2,0.910,2023-11-20 11:05:55.885662,2023-11-20 11:06:29.517902,0 days 00:00:33.632240,"{'activation': 'prelu', 'lin_prep_len': 1, 'li...",COMPLETE,GraphConv,prelu,1,0.5,False,512,[256],2,"{'mean': 64, 'max': 32, 'add': 16}",0.2,"{'mean': 32, 'max': 16, 'add': 8}",0.2,True
3,3,0.787,2023-11-20 11:06:29.520728,2023-11-20 11:06:56.705546,0 days 00:00:27.184818,NaN,COMPLETE,GraphConv,relu,2,0.492288,False,64,"[20, 16]",2,"{'mean': 8, 'max': 5, 'add': 2}",0.476272,"{'mean': 8, 'max': 7, 'add': 3}",0.035856,True
4,4,0.907,2023-11-20 11:06:56.707726,2023-11-20 11:07:31.458560,0 days 00:00:34.750834,NaN,COMPLETE,GraphConv,gelu,2,0.033315,False,934,"[586, 134]",2,"{'mean': 47, 'max': 10, 'add': 9}",0.308789,"{'mean': 39, 'max': 27, 'add': 4}",0.169701,True


In [48]:
runner2 = HypeRunner(data)

In [49]:
# you can optimize params with no dict, optuna makes it randomly
result2 = runner2.optimize_run(n_trials=3)

Sample data: 100%|██████████| 14/14 [00:00<00:00, 15.77it/s]
[I 2023-11-20 11:07:36,220] A new study created in memory with name: no-name-ca69d684-d645-47ca-937a-0ce95d9dfa29
/data/home/dapavlikov/coolgraph/cool_graph/runners.py:659: ExperimentalWarning: enqueue_trial is experimental (supported from v1.2.0). The interface can change in the future.
  self.study.enqueue_trial(trial_params)
/home/dapavlikov/.conda/envs/cool_graph_env2_py38/lib/python3.8/site-packages/optuna/study/study.py:857: ExperimentalWarning: create_trial is experimental (supported from v2.0.0). The interface can change in the future.
  create_trial(state=TrialState.WAITING, system_attrs={"fixed_params": params})
/home/dapavlikov/.conda/envs/cool_graph_env2_py38/lib/python3.8/site-packages/optuna/study/study.py:856: ExperimentalWarning: add_trial is experimental (supported from v2.0.0). The interface can change in the future.
  self.add_trial(
/home/dapavlikov/.conda/envs/cool_graph_env2_py38/lib/python3.8/site-packa

Study statistics: 
  Number of finished trials:  3
  Number of complete trials:  3
Best trial:
  Value:  0.924
  Params: 
{'conv_type': 'GraphConv', 'activation': 'leakyrelu', 'lin_prep_len': 1, 'lin_prep_dropout_rate': 0.4, 'lin_prep_weight_norm_flag': True, 'lin_prep_size_common': 512, 'lin_prep_sizes': [256], 'n_hops': 2, 'conv1_aggrs': {'mean': 128, 'max': 64, 'add': 32}, 'conv1_dropout_rate': 0.2, 'conv2_aggrs': {'mean': 64, 'max': 32, 'add': 16}, 'conv2_dropout_rate': 0.2, 'graph_conv_weight_norm_flag': True}


In [50]:
result2

,number,value,datetime_start,datetime_complete,duration,system_attrs_fixed_params,state,conv_type,activation,lin_prep_len,lin_prep_dropout_rate,lin_prep_weight_norm_flag,lin_prep_size_common,lin_prep_sizes,n_hops,conv1_aggrs,conv1_dropout_rate,conv2_aggrs,conv2_dropout_rate,graph_conv_weight_norm_flag
0,0,0.924,2023-11-20 11:07:36.222776,2023-11-20 11:08:08.834058,0 days 00:00:32.611282,"{'activation': 'leakyrelu', 'lin_prep_len': 1,...",COMPLETE,GraphConv,leakyrelu,1,0.4,True,512,[256],2,"{'mean': 128, 'max': 64, 'add': 32}",0.2,"{'mean': 64, 'max': 32, 'add': 16}",0.2,True
1,1,0.868,2023-11-20 11:08:08.836232,2023-11-20 11:08:37.657511,0 days 00:00:28.821279,NaN,COMPLETE,GraphConv,relu,2,0.492288,False,64,"[20, 16]",2,"{'mean': 8, 'max': 5, 'add': 2}",0.476272,"{'mean': 8, 'max': 7, 'add': 3}",0.035856,True
2,2,0.922,2023-11-20 11:08:37.660058,2023-11-20 11:09:14.773085,0 days 00:00:37.113027,NaN,COMPLETE,GraphConv,gelu,2,0.033315,False,934,"[586, 134]",2,"{'mean': 47, 'max': 10, 'add': 9}",0.308789,"{'mean': 39, 'max': 27, 'add': 4}",0.169701,True


# 6. Multitarget + groups of nodes (heterogeneous graph) in jupiter for Data <a name="p5"></a>

In [51]:
from cool_graph.runners import MultiRunner
import torch

In [52]:
data = torch.load("../tests/sample_data/sample_of_graph")

In [55]:
runner = MultiRunner(data)

In [56]:
result = runner.run()

Sample data: 100%|██████████| 288/288 [00:26<00:00, 10.99it/s]
2023-11-20 11:43:30.952 | INFO     | cool_graph.train.trainer:train:230 - 
Epoch 000: 
2023-11-20 11:43:54.536 | INFO     | cool_graph.train.helpers:eval_epoch:179 - test:
 {'label_3__accuracy__group_1': 0.928, 'label_3__accuracy__group_2': 0.944, 'label_3__cross_entropy__group_1': 0.257, 'label_3__cross_entropy__group_2': 0.208, 'label_3__f1_weighted__group_1': 0.894, 'label_3__f1_weighted__group_2': 0.917, 'label_3__roc_auc__group_1': 0.591, 'label_3__roc_auc__group_2': 0.69, 'label_4__accuracy__group_1': 0.952, 'label_4__accuracy__group_2': 0.969, 'label_4__cross_entropy__group_1': 0.194, 'label_4__cross_entropy__group_2': 0.13, 'label_4__f1_weighted__group_1': 0.928, 'label_4__f1_weighted__group_2': 0.954, 'label_4__roc_auc__group_1': 0.593, 'label_4__roc_auc__group_2': 0.747, 'label_5__accuracy__group_1': 0.944, 'label_5__accuracy__group_2': 0.96, 'label_5__cross_entropy__group_1': 0.204, 'label_5__cross_entropy__group

In [58]:
result["best_loss"]

{'label_3__accuracy__group_1': 0.928,
 'label_3__accuracy__group_2': 0.944,
 'label_3__cross_entropy__group_1': 0.257,
 'label_3__cross_entropy__group_2': 0.208,
 'label_3__f1_weighted__group_1': 0.894,
 'label_3__f1_weighted__group_2': 0.917,
 'label_3__roc_auc__group_1': 0.591,
 'label_3__roc_auc__group_2': 0.69,
 'label_4__accuracy__group_1': 0.952,
 'label_4__accuracy__group_2': 0.969,
 'label_4__cross_entropy__group_1': 0.194,
 'label_4__cross_entropy__group_2': 0.13,
 'label_4__f1_weighted__group_1': 0.928,
 'label_4__f1_weighted__group_2': 0.954,
 'label_4__roc_auc__group_1': 0.593,
 'label_4__roc_auc__group_2': 0.747,
 'label_5__accuracy__group_1': 0.944,
 'label_5__accuracy__group_2': 0.96,
 'label_5__cross_entropy__group_1': 0.204,
 'label_5__cross_entropy__group_2': 0.161,
 'label_5__f1_weighted__group_1': 0.916,
 'label_5__f1_weighted__group_2': 0.94,
 'label_5__roc_auc__group_1': 0.708,
 'label_5__roc_auc__group_2': 0.755,
 'label_6__accuracy__group_1': 0.98,
 'label_6__ac

In [59]:
result["test_metric"]

,label_3__accuracy__group_1,label_3__accuracy__group_2,label_3__cross_entropy__group_1,label_3__cross_entropy__group_2,label_3__f1_weighted__group_1,label_3__f1_weighted__group_2,label_3__roc_auc__group_1,label_3__roc_auc__group_2,label_4__accuracy__group_1,label_4__accuracy__group_2,...,label_6__accuracy__group_2,label_6__cross_entropy__group_1,label_6__cross_entropy__group_2,label_6__f1_weighted__group_1,label_6__f1_weighted__group_2,label_6__roc_auc__group_1,label_6__roc_auc__group_2,calc_time,main_metric,epoch
0,0.928,0.944,0.257,0.208,0.894,0.917,0.591,0.690,0.952,0.969,...,0.982,0.093,0.079,0.971,0.973,0.686,0.801,0.393,0.957,0
1,0.928,0.944,0.251,0.201,0.894,0.917,0.661,0.731,0.952,0.969,...,0.982,0.092,0.078,0.971,0.973,0.695,0.807,0.291,0.957,5
